In [1]:
# libraires for images and data
from PIL import Image
import re
import glob,os
from bs4 import BeautifulSoup
import cv2
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# libraries for deep learning
import torch
import torchvision
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
                                                                                                                                                                           
import logging
from Pegasus.api import *

from utils.wf import split_data_filenames, create_ann_list,create_augmented_filelist

In [ ]:
# When you restart the docker make sure to reinstall these:
! sudo python3 -m pip install --upgrade pip
! sudo python3 -m pip install --upgrade Pillow numpy torchvision matplotlib
! sudo python3 -m pip install pandas torch bs4

! sudo pip3 install opencv-python 
! sudo pip3 install --upgrade setuptools
! sudo pip3 install opencv-python 
! sudo pip3 install optuna==2.0.0 
! sudo pip3 install matplotlib 
! sudo pip3 install torch
! sudo pip3 install scikit-image
! sudo pip3 install torchvision 
! sudo pip3 install pytorchtools
! sudo pip3 install joblib

In [2]:
# DATA AQUSITION
imagesList = glob.glob('data/images/*.png')
annotationList = glob.glob('data/annotations/*.xml')

#DATA SPLIT
train_filenames,val_filenames,test_filenames, files_split_dict = split_data_filenames(imagesList)

#TODO: check the correctness of the fun
train_imgs, train_ann = create_ann_list(train_filenames)
val_imgs, val_ann     = create_ann_list(val_filenames)
test_imgs, test_ann   = create_ann_list(test_filenames)

In [3]:
#add images from group data
rc = ReplicaCatalog()

inputFiles = []
for img in imagesList:
    fileName = img.split("/")[-1]
    img_file = File(fileName)
    inputFiles.append(img_file)
    rc.add_replica("local", img_file,  os.path.join(os.getcwd(),str(img)))
    
annFiles = []
for ann in annotationList:
    fileName = ann.split("/")[-1]
    ann_file = File(fileName)
    annFiles.append(ann_file)
    rc.add_replica("local", ann_file,  os.path.join(os.getcwd(),str(ann)))
rc.write()

In [4]:
# Add Transformations

dist_plot = Transformation(
                "dist_plot",
                site="local",
                pfn=  os.path.join(os.getcwd(),"bin/plot_class_distribution.py"),
                is_stageable=True
            )

augment_imgs = Transformation(
                "augment_images",
                site="local",
                pfn=  os.path.join(os.getcwd(),"bin/data_aug.py"),
                is_stageable=True
            )

In [5]:
tc = TransformationCatalog()
tc.add_transformations(augment_imgs, dist_plot)#, model_hpo)
tc.write()

In [6]:
#Create workflow

wf = Workflow("mask_detection_workflow")

train_preprocessed_files = create_augmented_filelist(train_filenames,2)
distribution_plot_file = File("class_distribution.png")


# Add jobs to the workflow


# DATA EXPLORATION
# takes in all the annotationa files and creates plot with distribution of the classes

distribution_plot_job = Job(dist_plot)
distribution_plot_job.add_inputs(*train_ann, *val_ann, *test_ann)
distribution_plot_job.add_outputs(distribution_plot_file)


# DATA PREPROCESSING:TRAIN DATA-DATA AUGMENTATION
# takes images and adds gaussian noise to them

preprocess_train_job = Job(augment_imgs)
preprocess_train_job.add_inputs(*train_imgs)
preprocess_train_job.add_outputs(*train_preprocessed_files)

# DATA PREPROCESSING:VAL DATA-FILE RENAMING
#preprocess_val_job = Job(preprocess_imgs)
#preprocess_val_job.add_inputs(*val_imgs)
#preprocess_val_job.add_outputs(*val_preprocessed_files)

# DATA PREPROCESSING:TEST DATA-FILE RENAMING
#preprocess_test_job = Job(preprocess_imgs)
#preprocess_test_job.add_inputs(*test_imgs)
#preprocess_test_job.add_outputs(*test_preprocessed_files)


# HYPERPARAMETER OPTIMIZATION

# MODEL TRAINING

# MODEL EVALUATION


# INFERENCE
# takes images of our labmates and classifies them



In [7]:
# add jobs to workflow
wf.add_jobs(
    preprocess_train_job,
    distribution_plot_job
)

# run workflow
try:
    wf.plan(submit=True)
    wf.wait()
    wf.statistics()
except PegasusClientError as e:
    print(e.output)

[##################################################] 100.0% ..Success (Completed: 10, Queued: 0, Running: 0, Failed: 0)
